In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:10] 

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

In [ ]:
'''
we iterate through the list of words, and for each word we iterate through the string by 2, but before we create special start 
and end tokens to differentiate between words and to know which character is at the start and at the end. In order to learn the
statistics about which characters are likely to follow other characters, the simplest way in the bigram language models is to 
simply do it by counting. We are going to count how often any one of these combinations occurs in the training set.

'''

b = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
        # print(ch1, ch2)

In [ ]:
''' 
b.items return the tuples of ch bigrams and counts. sorted by default sorts by the first value of the tuple
to sort by the key we use lambda! takes the key value and returns kv at 1 (not 0 but at 1, which is the count) 
and - to go backwards
'''
sorted(b.items(), key = lambda kv: -kv[1])

In [ ]:
import torch

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

In [ ]:
'''
the characters are strings, but we have to index into an array using integers, so we need a lookup table 
from characters to integers

we concatenate words using join with '', this returns the entire dataset as a massive string. passing it to the set constructor
throws out duplicates from our jointed string, because sets do not allow duplicates. so it will be the set of the lowercase
characters. we dont actually want a set we want a list, then we sort it from a to z.

now we want the mentioned lookup table

we create the stoi variable (string to int). enumerate give us the iterator over the integer index and the actual element of the
list and then we are mapping the character to the integer

the special token have position 0, so we offset the other characters by +1.
'''

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');



In [ ]:
'''
after we create the array we have all the information necessary for us to actually sample from this bigram character level model
and roughly what we are going to do is just start following the probabilities and counts. In the beggining we start with 
the dot (our start token) so to sample the first character of a name we are looking at the first row, we have got 
the raw counts, then we need to convert to probabilities

'''

N[0]

In [ ]:
'''

we convert to float because we are going to normalize the counts

'''


p = N[0].float()
p = p / p.sum()
p

In [ ]:
'''

now we can sample from these distributions! we are going to use:
https://docs.pytorch.org/docs/stable/generated/torch.multinomial.html

roughly its is: you give me probabilities and i will give you integers, which are sampled according with the distribution.

to make it deterministic we will use a generator: https://docs.pytorch.org/docs/stable/generated/torch.Generator.html

'''

g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

In [ ]:
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

In [ ]:
'''
the matrix P will be similar to matrix N of counts, but every single row will have the probabilities normalized to 1, which 
indicates the probabily distribution of the next character given the character before it, which is defined by which row we are in.

the first reason is efficiency, the second reason for this is to practice the n-dimensional tensors, their manipulation and
something called broadcasting

the first step is to grab the fp copy of N and divide all the rows such that they sum to 1

but we need to be careful!

P.sum() sums up all the counts of the entire matrix N and give us a single number of everything. that's not what we want divide.
we want to simultaneously and in parallel divide all the rows by their respective sum.

in order to do that we read the docs: https://docs.pytorch.org/docs/stable/generated/torch.sum.html

torch.sum(input, dim, keepdim=False, *, dtype=None) → Tensor

we provide an input array and the dim (dimension) that we want to sum, in particular we want to sum up over rows.

one more argument to be careful is keepdim.

If keepdim is True, the output tensor is of the same size as input except in the dimension(s) dim where it is of size 1. 
Otherwise, dim is squeezed (see torch.squeeze()), resulting in the output tensor having 1 (or len(dim)) fewer dimension(s).

if its false then torch.sum not only makes the sum and collapses dimensions to be size 1, but in addition it squeezes out that
dimension

notice that P.shape is 27x27, so summing up across axis 0, we would be taking the zeroth dimension and summing across it, so when keepdim=True
then we not only get the counts along the columns but also the shape of this is 1x27, a row vector. Again, the reason we get the row vector is that
we passed in the 0 dimension and this 0 dimension becomes 1, and we've done a sum and we get a row. So basically we've done the sum vertically and
arrived just a single 1x27 vector of counts.

when we take out keepdim we just get 27, so it squeezes out that dimension and we just get a one dimensional vector of size 27

we dont want 1x27 because it gives us the sum of counts across the columns, we want to sum the other way, along dimension 1, with the shape 27x1, so
its a column vector. We are going  horizontally

'''

P.sum(1, keepdim=True).shape

In [ ]:
'''

now pay attention. its possible to take a 27x27 array and divide by a 27x1 array?

whether or not we can do this is determined by what's called broadcasting rules: https://docs.pytorch.org/docs/stable/notes/broadcasting.html

you'll notice that there is this special definition for what's called broadcasting that for whether or not these two arrays
can be combined in a binary op like division.
Two tensors are “broadcastable” if the following rules hold:
 
    Each tensor has at least one dimension (which is the case for us )

    When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal,
    one of them is 1, or one of them does not exist

so lets see if the second rule holds. 

we need to align the two arrays and their shapes

27 27
27  1

and we iterate from right to left. Each dimension must be either equal, one of them is 1, or one of them does not exist.
In this case they are not equal but one of them is one, so its fine
and the next iteration, they are both equal, so its fine too

so all the dimensions are fine, therefore this operation is broadcastable. This operation is allowed but what does these 
arrays do when we divide 27 by 1. It takes this dimension 1 and stretches it out, copies it to match 27. So it takes this
27x1 and copies it 27 times, in order to be 27x27 internally and does an element-wise division, which is what we want.

So we expect it to normalize every single row!

if we didn't use keepdim, we would be normalizing the columns, vertically

we have to respect broadcasting, make sure it is broadcasting in the right direction, otherwise you will introduce very subtle
bugs

'''


P = N.float()
#inplace operation in order to not create a new tensor
P /= P.sum(1, keepdim=True)

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    
    out = []
    ix = 0
    while True:
        #trained in bigrams
        #broadcast version
        p = P[ix]
        # here we keep fetching a row of N, converting to float and dividing every single iteration, which is expensive, thats why we have the P matrix
        #p = N[ix].float()
        #p = p / p.sum()

        #untrained
        # p = torch.ones(27) / 27.0
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

In [ ]:
'''

then after learning about internals of broadcast and vector normalization, we trained a bigram language model by just 
counting how frequently any pairing occurs and then normalizing so that we get a nice property distribution. So really the 
elements of array P are really the parameters of our bigram language model, giving us and summarizing the statistics. So 
we trained and know how to sample from the model, we iteratively just sample from next character and feed it in each time 
and get a next character.

now to summarize the quality of this model into a single number, how good it predicts the training set, for example. We can
now evaluate the training loss, which tells us the quality of this model, just like we saw in micrograd.

we use the log likelihood, which is the product of all probabilities inserted in a log func. the reason to use it is for
convenience bc we have mathematically that:

log(a*b*c) = log(a) + log(b) + log(c)

the likelihood is the product of probabilities and the log likelihood is just the sum of the logs of the individual 
probabilities 

so how high can log likelihood get? when all probabilities are 1, log likelihood will be zero and then when all probabilities
are lower, this will grow more and more negative. We don't actually like this because what we'd like is a loss function, and
the loss function has the semantics that low is good, because we are trying to minimize the loss, so we actually need to
invert this, which turns into negative loglikelihood.

nll is a very nice loss function because the lowest it can get is zero and the higher the worst the predictions are

'''
log_likelihood = 0.0
for w in words[:3]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
